## Downloading needed dependencies

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

!conda install -c conda-forge geopandas --yes
import geopandas as gpd

import os
import types
import pandas as pd
from botocore.client import Config
# import ibm_boto3

def __iter__(self): return 0


print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


## Download Data

In [36]:
# assumes data files are uploaded to git
data_urls = {
    'tallinn_districts': 'https://raw.githubusercontent.com/Marimais/Coursera_Capstone/master/data/tallinn_districts.geojson',
    'postal_codes': 'https://raw.githubusercontent.com/Marimais/Coursera_Capstone/master/data/sihtnumbrid.csv',
    'kpis': 'https://raw.githubusercontent.com/Marimais/Coursera_Capstone/master/data/kpis.csv',
    'districts': 'https://raw.githubusercontent.com/Marimais/Coursera_Capstone/master/data/District.csv'
}
data = {}
for name, url in data_urls.items():
    result = requests.get(url)
    if url.endswith('json'):
        data[name] = result.json()
    else:
        data[name] = result.text
    print(f'Downloaded {name} with content size {len(result.text)}')

Downloaded tallinn_districts with content size 461514
Downloaded postal_codes with content size 26472116
Downloaded kpis with content size 136811
Downloaded districts with content size 399


In [44]:
from io import StringIO
# Create dataframes
df_areas = pd.read_csv(StringIO(data['districts']))
df_areas.head()

,District,Latitude,Longitude,Area
0,..Haabersti city district,59.4267,24.6313,18.60
1,..Kesklinna city district,59.4328,24.7629,30.60
2,..Kristiine city district,59.4164,24.7100,9.40
3,..Lasnamäe city district,59.4293,24.8352,30.00
4,..Mustamäe city district,59.4010,24.6945,8.15


In [46]:
df_districts = pd.read_csv(StringIO(data['kpis']))
df_districts.head()

,DIM1,Place of residence,DIM2,Indicator,Value,Flag Codes,Flags
0,EE00,Whole country,N1,Number of families,359738.00,NaN,NaN
1,EE00,Whole country,N2,Number of family members,986799.00,NaN,NaN
2,EE00,Whole country,N3,Average size of family,2.74,NaN,NaN
3,EE00,Whole country,N4,Population,1294455.00,NaN,NaN
4,EE00,Whole country,N5,Share of population living in families,76.20,NaN,NaN


In [33]:
df_postal_codes =  pd.read_csv(StringIO(data['postal_codes']))
df_postal_codes.head()

,ADR_ID,SIHTNUMBER,LAHIAADRESS,Maakond,Vald,Linnaosa,VIITEPUNKT_X,VIITEPUNKT_Y
0,2378612,13516,Saina tn 1a,Harju maakond,Tallinn,..Haabersti city district,532761.39,6589855.63
1,2440202,10411,Kungla tn 30,Harju maakond,Tallinn,..Pohja-Tallinna city district,541379.74,6590130.75
2,2897069,10919,Kivimae tn 4/1,Harju maakond,Tallinn,..Nomme city district,537313.74,6582424.11
3,2468347,10125,L. Koidula tn 22d,Harju maakond,Tallinn,..Kesklinna city district,544363.07,6589050.54
4,2498165,10912,Toome pst 13d,Harju maakond,Tallinn,..Nomme city district,535518.59,6581234.11


## Data Shaping

In [47]:
# delete extra collummns and flags
df_districts.drop(['DIM1', 'DIM2','Flag Codes','Flags'], axis=1, inplace =True)
df_districts.head(2)

,Place of residence,Indicator,Value
0,Whole country,Number of families,359738.0
1,Whole country,Number of family members,986799.0


In [48]:
# rename column'Place of residence' as District
df_districts=df_districts.rename(columns={'Place of residence': 'District'})
df_districts.head(2)

,District,Indicator,Value
0,Whole country,Number of families,359738.0
1,Whole country,Number of family members,986799.0


In [49]:
# remove extra charachers and text from column 'District'
df_districts['District'] = df_districts['District'].map(lambda x: x.lstrip('.'))
df_districts['District'] = df_districts['District'].str.replace('city district','')
df_districts['District'] = df_districts['District'].str.strip()


df_areas['District'] = df_areas['District'].map(lambda x: x.lstrip('.'))
df_areas['District'] = df_areas['District'].str.replace('city district','')
df_areas['District']=df_areas['District'].str.strip()
df_districts.head(2)

,District,Indicator,Value
0,Whole country,Number of families,359738.0
1,Whole country,Number of family members,986799.0


In [6]:
# TODO
# set place of residence and indicator as indexes
#df_districts.set_index(['Place of residence', 'Indicator'],inplace=True)

In [7]:
# set District as index
#df_district_area.set_index(['District'],inplace=True)


In [8]:
#Calculate Share of families with children by deviding'Number of families with children aged under 18' by  and 'Number of families'
#print(df_districts.index.levels[0])
#for region in df_districts.index.levels[0]:
#    print(region)
#    df_districts.loc[(region, 'Newrow'), 'Value'] = 0
    


# for index, row in df_districts.iterrows():
#     print(index)
#     df_districts.loc[(index[0],'Share of families with children')]=(df_districts.loc[(index[0], 'Number of families with children aged under 18')]/df_districts.loc[(index[0], 'Number of families')] *100)

In [9]:
# df_districts.set_index('Newrow',append=True,inplace=True)
#df_districts.reset_index(inplace=True)
#df_districts.sort_values(by=['Place of residence', 'Indicator'])
# df_districts.order()
# df_districts.set_index(['Place of residence', 'Indicator'],inplace=True)

## Find out Tallinn coordinate

In [52]:
address = 'Tallinn, Estonia'

geolocator = Nominatim(user_agent="tln_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print(f'The geographical coordinate of Tallinas are {latitude}, {longitude}')

# create map of New York using latitude and longitude values
map_Tallinn = folium.Map(location=[latitude, longitude], zoom_start=12)


The geographical coordinate of Tallinas are 59.4250354, 24.74386926051089


## Add districts layer on the map

In [62]:
# Drawing Tallinn map with district layers, weighed by 'Share of population living in families'
data_map = folium.Map(location=[latitude, longitude], zoom_start=12)

df_map=df_districts[df_districts.Indicator=='Share of population living in families']

data_map.choropleth(geo_data=data['tallinn_districts'],
                   data=df_map,
                   columns=['District','Value'],
                   key_on='feature.properties.District',
                   fill_color='BuPu',
                   fill_opacity=0.5,
                   line_opacity=0.7,
                   legend_name='Share of population living in families'
                   )
res = folium.LayerControl().add_to(data_map)

In [63]:
# creating marker with additional information for each District
for i in range(df_areas.shape[0]):
    location = [df_areas['Latitude'][i],df_areas['Longitude'][i]]
    tooltip = "District:{}<br> Click for more".format(df_areas["District"][i])

    folium.Marker(location, # adding more details to the popup screen using HTML
                  popup="""
                  <i>Share of population living in families: </i> <br> <b>{}</b> <br> 
                  <i>Number of families with children aged under 18: </i><b><br>{}</b><br>
                  <i>Average size of family: </i><b><br>{}</b><br>""".format(
                    round(df_districts[(df_districts.District==df_areas["District"][i]) & (df_districts.Indicator=='Share of population living in families')].Value.values[0],1), 
                    round(df_districts[(df_districts.District==df_areas["District"][i]) & (df_districts.Indicator=='Number of families with children aged under 18')].Value.values[0],1), 
                    round(df_districts[(df_districts.District==df_areas["District"][i]) & (df_districts.Indicator=='Average size of family')].Value.values[0],1)),
                  icon=folium.Icon(color='red', icon='info-sign')).add_to(data_map)

In [64]:
data_map

## Define Foursquare Credentials and Version

In [14]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [15]:
CLIENT_ID = 'R2DI0ES2KM4GH4BJFR1AY102DHGGAF4WPBGTITH5BCRE0HV5' # your Foursquare ID
CLIENT_SECRET = 'HYUY25VXAZXK4XHSQCBOVQLR50APA4DCIIDJMFH0EPQ2ICC4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


In [41]:
LIMIT = 1000
radius = 10000

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID, 
CLIENT_SECRET, 
VERSION, 
latitude, 
longitude,
radius, 
LIMIT)

results = requests.get(url).json()
venues = results['response']['groups'][0]['items']
if not venues:
    continue
nearby_venues = json_normalize(venues) # flatten JSON
    
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.city','venue.location.lat', 'venue.location.lng','venue.postalCode']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]


SyntaxError: 'continue' not properly in loop (cell_name, line 22)

In [21]:
"""import math

LIMIT = 1000
df_venues = pd.DataFrame()

for i in range(df_district_area.shape[0]):
    radius = math.sqrt(df_district_area['Area'][i] * 1000/3.14)
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    df_district_area['Latitude'][i], 
    df_district_area['Longitude'][i], 
    radius, 
    LIMIT)

    results = requests.get(url).json()
    venues = results['response']['groups'][0]['items']
    if not venues:
        continue
    nearby_venues = json_normalize(venues) # flatten JSON
    
    # filter columns
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.city','venue.location.lat', 'venue.location.lng','venue.postalCode']
    nearby_venues =nearby_venues.loc[:, filtered_columns]

    # filter the category for each row
    nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

    # clean columns
    nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

#     nearby_venues['District']=df_district_area['District'][i]
    #df_venues=nearby_venues
  
    df_venues=df_venues.append(nearby_venues, ignore_index=True)
"""

In [24]:
del df_venues

In [94]:
df_district_area

,District,Latitude,Longitude,Area
0,Haabersti,59.4267,24.6313,18.60
1,Kesklinna,59.4328,24.7629,30.60
2,Kristiine,59.4164,24.7100,9.40
3,Lasnamäe,59.4293,24.8352,30.00
4,Mustamäe,59.4010,24.6945,8.15
5,Nõmme,59.3807,24.6995,28.70
6,Pirita,59.4749,24.8725,18.70
7,Põhja-Tallinna,59.4550,24.6894,17.30


In [126]:
!conda install -c conda-forge geopandas --yes
import geopandas as gpd

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopandas-0.6.2            |             py_0         906 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.0 MB

The following packages will be UPDATED:

    ca-certificates: 2019.11.27-0      --> 2019.11.28-hecc5488_0 conda-forge
    certifi:         2019.11.28-py36_0 --> 2019.11.28-py36_0     conda-forge
    geopandas:       0.6.1-py_0        --> 0.6.2-py_0            conda-forge

The following packages will be DOWNGRADED:

    openssl:         1.1.1d-h7b6447c_3 --> 1.1.1d-h516909a_0     conda-forge


geopandas-0.6.2      | 906 KB   

In [128]:
df_venues['geometry'] = gpd.points_from_xy(df_venues['lat'], df_venues['lng'])
gdf_venues = gpd.GeoDataFrame(df_venues)

pointInPolys = gpd.tools.sjoin(gdf_venues, districts_gdf, how='left')
pointInPolys

TypeError: 'module' object is not callable

In [39]:
#del df_venues
# Shopping Mall, Supermarket, Grocery Store, Market, Playground, Park, Trail, Beach, Bus Station, Movie Theater, Zoo

In [22]:
df=df_venues.groupby(['categories'])
df.first() 


,name,city,lat,lng,District
categories,,,,,
Airport,Tallinn Airport (TLL) (Lennart Meri Tallinna L...,Tallinn,59.416381,24.799472,Lasnamäe
Airport Terminal,Check In,Tallinn,59.416289,24.799117,Lasnamäe
American Restaurant,Mack Bar-B-Que,Tallinn,59.426536,24.652412,Haabersti
Animal Shelter,Loomade Hoiupaik,Tallinn,59.372065,24.741690,Nõmme
Arcade,Happy Fly batuudikeskus,Tallinn,59.401472,24.656382,Haabersti
Art Gallery,Fotografiska,Tallinn,59.438230,24.728266,Kristiine
Art Museum,Kumu kunstimuuseum (Kumu Kunstimuuseum),Tallinn,59.436186,24.796192,Lasnamäe
Arts & Crafts Store,Kangas ja Nööp,Tallinn,59.412599,24.667962,Nõmme
Asian Restaurant,Chow king,Tallinn,59.410471,24.682543,Haabersti


In [34]:
# find out corallation for parks and number of families in District
fam_venues = pd.DataFrame()

# first lets filter out posible important venues for families 
for i in range(df_district_area.shape[0]):
    fam_venues = fam_venues.append(df_venues[df_venues['categories']=='Park'], ignore_index=True)

In [35]:
fam_venues

,name,categories,city,lat,lng,District
0,Glehni park,Park,Tallinn,59.387104,24.653071,Haabersti
1,Sütiste parkmets,Park,Tallinn,59.395993,24.691840,Haabersti
2,Männi park,Park,Tallinn,59.398258,24.676693,Haabersti
3,Parditiigi park,Park,NaN,59.405276,24.704353,Haabersti
4,Löwenruh' park,Park,Tallinn,59.417502,24.695426,Haabersti
5,Kopli Kalmistupark,Park,Tallinn,59.453659,24.684674,Haabersti
6,Sütiste parkmets,Park,Tallinn,59.395993,24.691840,Kristiine
7,Schnelli Park,Park,Tallinn,59.439098,24.736645,Kristiine
8,Harju tänava park,Park,Tallinn,59.435215,24.743514,Kristiine
9,Kadrioru park,Park,Tallinn,59.440523,24.794699,Lasnamäe


In [33]:
del fam_venues

In [23]:
df_venues[df_venues['name']=='Glehni park']

,name,categories,city,lat,lng,postalCode
